In [10]:
### Load and preprocess the dataset

import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tqdm import tqdm 

image_height = 22
image_width = 54
batch_size = 32

image_data = []
labels = []

image_folder = "./images_w_labels"
for filename in tqdm(os.listdir(image_folder)):
    if filename.endswith(".jpg"):
        img = load_img(os.path.join(image_folder, filename), target_size=(image_height, image_width))
        img_array = img_to_array(img)
        image_data.append(img_array)

        # Extract the labels from the file name
        label = filename.split(".")[0]  # Remove the file extension
        labels.append([int(digit) for digit in label])

image_data = tf.convert_to_tensor(image_data)
labels = tf.convert_to_tensor(labels)

dataset = tf.data.Dataset.from_tensor_slices((image_data, labels))
dataset = dataset.shuffle(len(image_data))
dataset = dataset.batch(batch_size)


100%|██████████| 2466/2466 [00:01<00:00, 1844.84it/s]


In [11]:
# Define the split ratios
train_ratio = 0.7  # 70% of data for training
val_ratio = 0.15  # 15% of data for validation
test_ratio = 0.15  # 15% of data for testing

# Split the dataset
train_size = int(train_ratio * len(dataset))
val_size = int(val_ratio * len(dataset))
test_size = int(test_ratio * len(dataset))

train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size).take(val_size)
test_dataset = dataset.skip(train_size + val_size).take(test_size)

In [12]:
for i in train_dataset:
    print(i[0].shape, i[1].shape)
    break

(32, 22, 54, 3) (32, 4)


In [13]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from functools import partial

DefaultConv2D = partial(Conv2D, kernel_size=3, padding="same", activation="relu", kernel_initializer="he_normal")
model = tf.keras.Sequential([
	DefaultConv2D(filters=64, kernel_size=7, input_shape=(22, 54, 3)),
	MaxPool2D(),
	DefaultConv2D(filters=128),
	DefaultConv2D(filters=128),
	MaxPool2D(),
	DefaultConv2D(filters=256),
	DefaultConv2D(filters=256),
	MaxPool2D(),
	Flatten(),
	Dense(units=128, activation="relu", kernel_initializer="he_normal"),
	Dropout(0.5),
	Dense(units=64, activation="relu", kernel_initializer="he_normal"),
	Dropout(0.5),
	Dense(units=4, activation="softmax")
])

# # Model Architecture
# model = tf.keras.Sequential([
#     layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(22, 54, 3)),
#     layers.MaxPooling2D(pool_size=(2, 2)),
#     layers.Flatten(),
#     layers.Dense(128, activation='relu'),
#     layers.Dense(4, activation='softmax')
# ])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
history = model.fit(train_dataset, epochs=epochs, validation_data=val_dataset)

Epoch 1/10
54/54 [==============================] - 2s 29ms/step - loss: 566196.3125 - accuracy: 0.3003 - val_loss: 1860945.5000 - val_accuracy: 0.2756
Epoch 2/10
54/54 [==============================] - 1s 24ms/step - loss: 5365002.0000 - accuracy: 0.2865 - val_loss: 10786529.0000 - val_accuracy: 0.2585
Epoch 3/10
54/54 [==============================] - 1s 26ms/step - loss: 18176414.0000 - accuracy: 0.2697 - val_loss: 27491934.0000 - val_accuracy: 0.2898
Epoch 4/10
54/54 [==============================] - 1s 25ms/step - loss: 39586540.0000 - accuracy: 0.2940 - val_loss: 49026176.0000 - val_accuracy: 0.3011
Epoch 5/10
54/54 [==============================] - 1s 25ms/step - loss: 66583988.0000 - accuracy: 0.2789 - val_loss: 91884952.0000 - val_accuracy: 0.3125
Epoch 6/10
54/54 [==============================] - 1s 25ms/step - loss: 105073928.0000 - accuracy: 0.2784 - val_loss: 118630760.0000 - val_accuracy: 0.2472
Epoch 7/10
54/54 [==============================] - 1s 27ms/step - loss:

In [ ]:
# 4. Model Evaluation
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test Accuracy:', test_acc)

In [ ]:

# 5. Prediction
predictions = model.predict(new_images)
